In [1]:
import torch 
import spacy
from datasets import load_dataset
from tqdm import tqdm
import os

In [2]:
def get_entities(nlp,doc):
    doc=doc.replace('(CNN)','')
    s = nlp(doc)
    all_entities = [ent.text for ent in s.ents]
    return all_entities
def approximate_match_number(entity_list1,entity_list2):
    match_num=0
    for e1 in entity_list1:
        for e2 in entity_list2:
            if e1.lower()==e2.lower() or e1.replace('the','').strip()==e2 or e1==e2.replace('the','').strip():
                match_num+=1
                break
    return match_num
            

In [3]:
dataset = load_dataset(
        "cnn_dailymail", "2.0.0", cache_dir="/scratch/wenxiao/topic_model/dataset/"
    )
# dataset = load_dataset(
#         "xsum",  cache_dir="/scratch/wenxiao/topic_model/dataset/"
#     )
# dataset = load_dataset(
#         "scientific_papers", "arxiv", cache_dir="/scratch/wenxiao/topic_model/dataset/"
#     )
dataset = load_dataset(
        "multi_news", cache_dir="/scratch/wenxiao/topic_model/dataset/"
    )
nlp = spacy.load("en_core_web_sm")

Downloading:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/918 [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/257M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset multi_news downloaded and prepared to /scratch/wenxiao/topic_model/dataset/multi_news/default/1.0.0/2e145a8e21361ba4ee46fef70640ab946a3e8d425002f104d2cda99a9efca376. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# Analyze the entities between doc and summ in CNNDM

In [4]:
dataset = load_dataset(
        "cnn_dailymail", "2.0.0", cache_dir="/scratch/wenxiao/topic_model/dataset/"
    )

dict_keys(['article', 'highlights', 'id'])

In [16]:
recall=[]
precision=[]
f1=[]
doc_entity_num=[]
summ_entity_num=[]
match_num=[]
for i in tqdm(range(len(dataset['validation']))):
    doc_entities = set(get_entities(nlp,dataset['validation'][i]['article']))
    doc_entity_num.append(len(doc_entities))

    summ_entities = set(get_entities(nlp,dataset['validation'][i]['highlights']))
    summ_entity_num.append(len(summ_entities))
    
    num_intersect=approximate_match_number(doc_entities,summ_entities)
    match_num.append(num_intersect)
#     num_intersect = len(doc_entities.intersection(summ_entities))
    r= num_intersect/float(len(summ_entities)) if len(summ_entities)!=0 else 0
    p= num_intersect/float(len(doc_entities)) if len(doc_entities)!=0 else 0
    f = 2*p*r/(p+r) if r!=0 or p!=0 else 0

    recall.append(r)
    precision.append(p)
    f1.append(f)
    

100%|██████████| 13368/13368 [19:32<00:00, 11.40it/s]


In [17]:
print(sum(precision)/len(precision))
print(sum(recall)/len(recall))
print(sum(f1)/len(f1))
print(sum(match_num)/len(match_num))
print(sum(doc_entity_num)/len(doc_entity_num))
print(sum(summ_entity_num)/len(summ_entity_num))

0.1605695594372795
0.8315119888856853
0.2580228363723967
5.557824655894674
40.682675044883304
6.662477558348295


# Analyze the entities between doc and summ in Pubmed

In [18]:
dataset = load_dataset(
        "scientific_papers", "pubmed", cache_dir="/scratch/wenxiao/topic_model/dataset/"
    )

Reusing dataset scientific_papers (/scratch/wenxiao/topic_model/dataset/scientific_papers/pubmed/1.1.1/306757013fb6f37089b6a75469e6638a553bd9f009484938d8f75a4c5e84206f)


  0%|          | 0/3 [00:00<?, ?it/s]

In [19]:
dataset['validation'][0].keys()

dict_keys(['article', 'abstract', 'section_names'])

In [20]:
recall=[]
precision=[]
f1=[]
doc_entity_num=[]
summ_entity_num=[]
match_num = []
for i in tqdm(range(len(dataset['validation']))):
    doc_entities = set(get_entities(nlp,dataset['validation'][i]['article']))
    doc_entity_num.append(len(doc_entities))

    summ_entities = set(get_entities(nlp,dataset['validation'][i]['abstract']))
    summ_entity_num.append(len(summ_entities))
    
    num_intersect=approximate_match_number(doc_entities,summ_entities)
#     num_intersect = len(doc_entities.intersection(summ_entities))

    
    r= num_intersect/float(len(summ_entities)) if len(summ_entities)!=0 else 0
    p= num_intersect/float(len(doc_entities)) if len(doc_entities)!=0 else 0
    f = 2*p*r/(p+r) if r!=0 or p!=0 else 0

    recall.append(r)
    precision.append(p)
    f1.append(f)
    match_num.append(num_intersect)

100%|██████████| 6633/6633 [40:26<00:00,  2.73it/s]   


In [21]:
print(sum(precision)/len(precision))
print(sum(recall)/len(recall))
print(sum(f1)/len(f1))
print(sum(match_num)/len(match_num))
print(sum(doc_entity_num)/len(doc_entity_num))
print(sum(summ_entity_num)/len(summ_entity_num))

0.07494208263931407
0.6407041769589116
0.1249531545748999
4.62792100105533
72.1676466154078
6.429519071310116


# Analyze the entities between doc and summ in arXiv

In [22]:
dataset = load_dataset(
        "scientific_papers", "arxiv", cache_dir="/scratch/wenxiao/topic_model/dataset/"
    )

Reusing dataset scientific_papers (/scratch/wenxiao/topic_model/dataset/scientific_papers/arxiv/1.1.1/306757013fb6f37089b6a75469e6638a553bd9f009484938d8f75a4c5e84206f)


  0%|          | 0/3 [00:00<?, ?it/s]

In [23]:
recall=[]
precision=[]
f1=[]
doc_entity_num=[]
summ_entity_num=[]
match_num = []
for i in tqdm(range(len(dataset['validation']))):
    doc_entities = set(get_entities(nlp,dataset['validation'][i]['article']))
    doc_entity_num.append(len(doc_entities))

    summ_entities = set(get_entities(nlp,dataset['validation'][i]['abstract']))
    summ_entity_num.append(len(summ_entities))
    
    num_intersect=approximate_match_number(doc_entities,summ_entities)
#     num_intersect = len(doc_entities.intersection(summ_entities))

    r= num_intersect/float(len(summ_entities)) if len(summ_entities)!=0 else 0
    p= num_intersect/float(len(doc_entities)) if len(doc_entities)!=0 else 0
    f = 2*p*r/(p+r) if r!=0 or p!=0 else 0

    recall.append(r)
    precision.append(p)
    f1.append(f)
    match_num.append(num_intersect)
    

100%|██████████| 6436/6436 [1:15:42<00:00,  1.42it/s]


In [24]:
print(sum(precision)/len(precision))
print(sum(recall)/len(recall))
print(sum(f1)/len(f1))
print(sum(match_num)/len(match_num))
print(sum(doc_entity_num)/len(doc_entity_num))
print(sum(summ_entity_num)/len(summ_entity_num))

0.019889883603868053
0.5860607551957498
0.03677752806125409
1.973586078309509
144.79039776258546
3.11544437538844


# Analyze the entities between doc and summ in Multi-News

In [25]:
dataset = load_dataset(
        "multi_news", cache_dir="/scratch/wenxiao/topic_model/dataset/"
    )

Using custom data configuration default
Reusing dataset multi_news (/scratch/wenxiao/topic_model/dataset/multi_news/default/1.0.0/2e145a8e21361ba4ee46fef70640ab946a3e8d425002f104d2cda99a9efca376)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
dataset['validation'][0].keys()

dict_keys(['document', 'summary'])

In [26]:
recall=[]
precision=[]
f1=[]
doc_entity_num=[]
summ_entity_num=[]
match_num = []
for i in tqdm(range(len(dataset['validation']))):
    doc_entities = set(get_entities(nlp,dataset['validation'][i]['document']))
    doc_entity_num.append(len(doc_entities))

    summ_entities = set(get_entities(nlp,dataset['validation'][i]['summary']))
    summ_entity_num.append(len(summ_entities))
    
    num_intersect=approximate_match_number(doc_entities,summ_entities)
#     num_intersect = len(doc_entities.intersection(summ_entities))
    
    r= num_intersect/float(len(summ_entities)) if len(summ_entities)!=0 else 0
    p= num_intersect/float(len(doc_entities)) if len(doc_entities)!=0 else 0
    f = 2*p*r/(p+r) if r!=0 or p!=0 else 0

    recall.append(r)
    precision.append(p)
    f1.append(f)
    match_num.append(num_intersect)

100%|██████████| 5622/5622 [23:13<00:00,  4.03it/s]   


In [27]:
print(sum(precision)/len(precision))
print(sum(recall)/len(recall))
print(sum(f1)/len(f1))
print(sum(match_num)/len(match_num))
print(sum(doc_entity_num)/len(doc_entity_num))
print(sum(summ_entity_num)/len(summ_entity_num))

0.17148039470515866
0.6563465701313178
0.2566763331499742
11.957310565635005
90.84204909284952
17.9062611170402


# Analyze the entities between doc and summ in XSUM

In [4]:
dataset['validation'][0].keys()

dict_keys(['document', 'summary', 'id'])

In [7]:
recall=[]
precision=[]
f1=[]
doc_entity_num=[]
summ_entity_num=[]
match_num = []
for i in tqdm(range(len(dataset['validation']))):
    doc_entities = set(get_entities(nlp,dataset['validation'][i]['document']))
    doc_entity_num.append(len(doc_entities))

    summ_entities = set(get_entities(nlp,dataset['validation'][i]['summary']))
    summ_entity_num.append(len(summ_entities))
    
    num_intersect=approximate_match_number(doc_entities,summ_entities)
#     num_intersect = len(doc_entities.intersection(summ_entities))
    r= num_intersect/float(len(summ_entities)) if len(summ_entities)!=0 else 0
    p= num_intersect/float(len(doc_entities)) if len(doc_entities)!=0 else 0
    f = 2*p*r/(p+r) if r!=0 or p!=0 else 0

    recall.append(r)
    precision.append(p)
    f1.append(f)
    match_num.append(num_intersect)
    break

  0%|          | 0/11332 [00:00<?, ?it/s]


In [6]:
print(sum(precision)/len(precision))
print(sum(recall)/len(recall))
print(sum(f1)/len(f1))
print(sum(match_num)/len(match_num))
print(sum(doc_entity_num)/len(doc_entity_num))
print(sum(summ_entity_num)/len(summ_entity_num))

0.04176096760600055
0.36260828877754414
0.07124630804607004
27.743293328626898
2.7198199788210378


In [18]:
set(get_entities(nlp,dataset['validation'][2]['document']))

{'100',
 '2012',
 'Alexandra Shulman',
 'British Vogue',
 'Diana',
 'Josh',
 'Josh Olins',
 'June',
 'Kensington Palace',
 'Nicholas Cullinan',
 'Norfolk',
 'Seven',
 'Vogue',
 'Wales',
 'Wednesday',
 'first',
 'four',
 'the Duchess of Cambridge',
 'the National Portrait Gallery',
 'the day',
 'the past 100 years',
 'two'}

In [19]:
set(get_entities(nlp,dataset['validation'][2]['summary']))


{'British Vogue', 'The Duchess of Cambridge'}

In [16]:
dataset['validation'][2]['document']

'Seven photographs taken in the Norfolk countryside by photographer Josh Olins will appear in the June edition.\nIn her first sitting for a magazine, the duchess is seen looking relaxed and wearing casual clothes.\nThe shoot was in collaboration with the National Portrait Gallery, where two images are being displayed in the Vogue 100: A Century of Style exhibition.\nThe duchess, who has a keen interest in photography, has been patron of the National Portrait Gallery since 2012.\nNicholas Cullinan, director of the National Portrait Gallery, said: "Josh has captured the duchess exactly as she is - full of life, with a great sense of humour, thoughtful and intelligent, and in fact, very beautiful."\nHe said the images also encapsulated what Vogue had done over the past 100 years - "to pair the best photographers with the great personalities of the day, in order to reflect broader shifts in culture and society".\nAlexandra Shulman, editor-in-chief of British Vogue, said: "To be able to pub

In [17]:
dataset['validation'][2]['summary']

"The Duchess of Cambridge will feature on the cover of British Vogue to mark the magazine's centenary."

# Analyze the entities in generated summary and gt summary for CNNDM

In [14]:
# pegasus_dir = '/scratch/wenxiao/topic_guided_summ/pegasus/generated_txt_0_cnndm_beam=1_1024_1024/'
# pegasus_dir = '/scratch/wenxiao/topic_guided_summ/pegasus_ent_a=1/generated_txt_0_cnndm_beam=1_1024_1024/'
pegasus_dir = '/scratch/wenxiao/topic_guided_summ/spancopy_newloss_cnndm/generated_txt_0_cnndm_beam=1_1024_1024/'

all_files = os.listdir(pegasus_dir)
all_files.sort()
recall=[]
precision=[]
f1=[]
generated_entity_num=[]
gt_entity_num=[]
for i,f in enumerate(tqdm(all_files)):
    with open(pegasus_dir+f) as of:
        generated_summ=of.read()
    fid = int(f[:-4])
    gt_summ = dataset['test'][fid]['highlights']
    generated_entities = set(get_entities(nlp,generated_summ))
    generated_entity_num.append(len(generated_entities))
    gt_entities = set(get_entities(nlp,gt_summ))
    gt_entity_num.append(len(gt_entities))
    num_intersect=approximate_match_number(generated_entities,gt_entities)
#     num_intersect = len(doc_entities.intersection(summ_entities))
    r= num_intersect/float(len(gt_entities)) if len(gt_entities)!=0 else 0
    p= num_intersect/float(len(generated_entities)) if len(generated_entities)!=0 else 0
    f = 2*p*r/(p+r) if r!=0 or p!=0 else 0

    recall.append(r)
    precision.append(p)
    f1.append(f)
#     break

100%|██████████| 11490/11490 [03:39<00:00, 52.46it/s]


In [15]:
print(sum(precision)/len(precision))
print(sum(recall)/len(recall))
print(sum(f1)/len(f1))
print(sum(gt_entity_num)/len(gt_entity_num))
print(sum(generated_entity_num)/len(generated_entity_num))

0.42697663005117004
0.4140268753530566
0.3974646503365596
6.418798955613577
6.060922541340296


# Analyze the entities in generated summary and gt summary for XSum

In [24]:
pegasus_dir = '/scratch/wenxiao/topic_guided_summ/pegasus_xsum/'

all_files = os.listdir(pegasus_dir)
all_files.sort()
recall=[]
precision=[]
f1=[]
generated_entity_num=[]
gt_entity_num=[]
for i,f in tqdm(enumerate(all_files)):
    with open(pegasus_dir+f) as of:
        generated_summ=of.read()
    fid = int(f[:-4])
    gt_summ = dataset['test'][fid]['summary']
    generated_entities = set(get_entities(nlp,generated_summ))
    generated_entity_num.append(len(generated_entities))
    gt_entities = set(get_entities(nlp,gt_summ))
    gt_entity_num.append(len(gt_entities))
    num_intersect=approximate_match_number(generated_entities,gt_entities)
#     num_intersect = len(doc_entities.intersection(summ_entities))
    r= num_intersect/float(len(gt_entities)) if len(gt_entities)!=0 else 0
    p= num_intersect/float(len(generated_entities)) if len(generated_entities)!=0 else 0
    f = 2*p*r/(p+r) if r!=0 or p!=0 else 0

    recall.append(r)
    precision.append(p)
    f1.append(f)
#     break

11334it [02:23, 78.99it/s]


In [25]:
print(sum(precision)/len(precision))
print(sum(recall)/len(recall))
print(sum(f1)/len(f1))
print(sum(gt_entity_num)/len(gt_entity_num))
print(sum(generated_entity_num)/len(generated_entity_num))

0.4506206650061949
0.41665846848483096
0.4154179812537753
2.7310746426680783
2.47335450855832


In [33]:
with open(pegasus_dir+all_files[0]) as of:
    generated_summ=of.read()

In [27]:
dataset['test'][fid]

{'document': "Giaccherini, 31, joined the Black Cats in 2013 from Juventus for £6.5m but, after an impressive first season, he struggled to make an impact.\nHe played 43 times in total for the club, scoring five goals, but spent last season on loan at Serie A Bologna.\nThe Italian has 29 international caps and scored the Azzurri's opening goal of Euro 2016 against Belgium.\nFind all the latest football transfers on our dedicated page.",
 'summary': 'Sunderland and Italy winger Emanuele Giaccherini has joined Napoli on a three-year deal for an undisclosed fee.',
 'id': '36815932'}